# Certified Finetuning of a Classifier on the OCT-MNIST Dataset

In [1]:
%load_ext autoreload
%autoreload 2
import torch
import tqdm
import abstract_gradient_training as agt
from abstract_gradient_training import AGTConfig
from abstract_gradient_training.certified_training import utils as ct_utils
from models.deepmind import DeepMindSmall 
from datasets import oct_mnist

## Pre-train the model

Exclude class 2 (Drusen) from the pretraining.

In [2]:
# set up pre-training
torch.manual_seed(0)
device = torch.device("cuda:0")
pretrain_batchsize = 100
pretrain_n_epochs = 20
pretrain_learning_rate = 0.001

In [3]:
# define model, dataset and optimizer
model = DeepMindSmall(1, 1)
dl_train, dl_test = oct_mnist.get_dataloaders(pretrain_batchsize, exclude_classes=[2])
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=pretrain_learning_rate)
model = model.to(device)
# model.load_state_dict(torch.load("medmnist.ckpt"))

/vol/bitbucket/ps1623/venv/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [4]:
# pre-train the model
progress_bar = tqdm.trange(pretrain_n_epochs, desc="Epoch", )
for epoch in progress_bar:
    for i, (x, u) in enumerate(dl_train):
        # Forward pass
        u, x = u.to(device), x.to(device)
        output = model(x)
        loss = criterion(output.squeeze().float(), u.squeeze().float())
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            progress_bar.set_postfix(loss=loss.item())

Epoch: 100%|██████████| 20/20 [00:41<00:00,  2.08s/it, loss=0.0275]


## Fine-tune the model

Include all classes, only allowing class 2 (Drusen) to be potentially poisoned.

In [5]:
# set up fine-tuning parameters
clean_batchsize = 7000
drusen_batchsize = 3000
test_batchsize = 1000

config = AGTConfig(
    fragsize = 2000,
    learning_rate = 0.6,
    n_epochs = 2,
    label_k_poison = 50,
    forward_bound = "interval",
    device = "cuda:0",
    backward_bound = "interval",
    loss = "binary_cross_entropy",
    lr_decay=2.0,
    lr_min=0.001,
)

In [6]:
# get dataloaders
dl_clean, dl_test_clean = oct_mnist.get_dataloaders(clean_batchsize, test_batchsize, exclude_classes=[2])
dl_drusen, dl_test_drusen = oct_mnist.get_dataloaders(drusen_batchsize, test_batchsize, exclude_classes=[0, 1, 3])
_, dl_test_all = oct_mnist.get_dataloaders(clean_batchsize, test_batchsize)

In [7]:
# evaluate the pre-trained model
param_n, param_l, param_u = ct_utils.get_parameters(model)
drusen_acc = agt.test_metrics.test_accuracy(param_n, param_l, param_u, dl_test_drusen, model, ct_utils.propagate_conv_layers)
clean_acc = agt.test_metrics.test_accuracy(param_n, param_l, param_u, dl_test_clean, model, ct_utils.propagate_conv_layers)
all_acc = agt.test_metrics.test_accuracy(param_n, param_l, param_u, dl_test_all, model, ct_utils.propagate_conv_layers)

print("=========== Pre-trained model accuracy ===========")
print(f"Class 2 (Drusen) : nominal = {drusen_acc[1]:.2g}")
print(f"Classes 0, 1, 3  : nominal = {clean_acc[1]:.2g}")
print(f"All Classes      : nominal = {all_acc[1]:.2g}")

=========== Pre-trained model accuracy ===========
Class 2 (Drusen) : nominal = 0.58
Classes 0, 1, 3  : nominal = 0.95
All Classes      : nominal = 0.86


In [8]:
# fine-tune the model using abstract gradient training (keeping the convolutional layers fixed)
param_l, param_n, param_u = agt.poison_certified_training(
    model, config, dl_drusen, dl_test_drusen, dl_clean=dl_clean, transform=ct_utils.propagate_conv_layers
)

[AGT] [INFO    ] [17:41:03] Starting Poison Certified Training


[AGT] [INFO    ] [17:41:03] Training batch 0: Network eval bounds=(0.58, 0.58, 0.58), W0 Bound=0.0 
[AGT] [INFO    ] [17:41:04] Training batch 1: Network eval bounds=(0.77, 0.85, 0.85), W0 Bound=0.251 
[AGT] [INFO    ] [17:41:06] Training batch 2: Network eval bounds=(0.74, 0.85, 0.87), W0 Bound=0.4 
[AGT] [INFO    ] [17:41:07] Training batch 3: Network eval bounds=(0.7 , 0.85, 0.89), W0 Bound=0.527 
[AGT] [INFO    ] [17:41:09] Finished Poison Certified Training



In [9]:
# evaluate the fine-tuned model
drusen_acc = agt.test_metrics.test_accuracy(param_n, param_l, param_u, dl_test_drusen, model, ct_utils.propagate_conv_layers)
clean_acc = agt.test_metrics.test_accuracy(param_n, param_l, param_u, dl_test_clean, model, ct_utils.propagate_conv_layers)
all_acc = agt.test_metrics.test_accuracy(param_n, param_l, param_u, dl_test_all, model, ct_utils.propagate_conv_layers)

print("=========== Fine-tuned model accuracy + bounds ===========")
print(f"Class 2 (Drusen) : nominal = {drusen_acc[1]:.2g}, certified bound = {drusen_acc[0]:.2g}")
print(f"Classes 0, 1, 3  : nominal = {clean_acc[1]:.2g}, certified bound = {clean_acc[0]:.2g}")
print(f"All Classes      : nominal = {all_acc[1]:.2g}, certified bound = {all_acc[0]:.2g}")

=========== Fine-tuned model accuracy + bounds ===========
Class 2 (Drusen) : nominal = 0.85, certified bound = 0.67
Classes 0, 1, 3  : nominal = 0.89, certified bound = 0.83
All Classes      : nominal = 0.88, certified bound = 0.79
